#**Машинное обучение ИБ-2024**

#**Домашнее задание 1.**
#Регрессия, KNN, LinearRegression.

В данной домашней работе мы будем строить модели для предсказания цены квартиры в России. Ниже приведено описание некоторых колонок набора данных.

date - дата публикации объявления

price - цена в рублях

level- этаж, на котором находится квартира

levels - количество этажей в квартире

rooms - количество комнат в квартире. Если значение -1, то квартира считается апартаментами.

area - площадь квартиры.

kitchen_area - площадь кухни.

geo_lat - Latitude

geo_lon - Longitude

building_type - материал застройки. 0 - Don't know. 1 - Other. 2 - Panel. 3 - Monolithic. 4 - Brick. 5 - Blocky. 6 - Wooden

#Часть 0. Начало работы

Для начала работы с данными импортируем библиотеки, которые понадобятся в данном задании.

In [ ]:
import math
import pandas as pd
import numpy as np
import matplotlib as plt
import sklearn
import seaborn as sns

Загрузим библиотеку folium для отображения данных на карте по координатам.

In [ ]:
!pip install folium

Распакуем наши данные из архива.

In [ ]:
!unzip ...
# Файл не был заархивирован, поэтому не нужная часть

"unzip" �� ���� ����७��� ��� ���譥�
��������, �ᯮ��塞�� �ணࠬ��� ��� ������ 䠩���.


Загрузим данные из csv файла в датафрейм.

In [2]:
df = pd.read_csv('input_data.csv', sep=';')
df
# В гуглколлаб возникала проблема с чтение файла: неправильно определялось количество строк, считывалось небольшое количество строк, 
# при повторениии этой части кода считывалось с каждым разом больше и и больше, но не всё, в vscode всё работало хорошо (не знаю зачем рассказал, но вот)

,date,price,level,levels,rooms,area,kitchen_area,geo_lat,geo_lon,building_type,object_type,postal_code,street_id,id_region,house_id
0,2021-01-01,2451300,15,31,1,30.3,0.0,56.780112,60.699355,0,2,620000.0,NaN,66,1632918.0
1,2021-01-01,1450000,5,5,1,33.0,6.0,44.608154,40.138381,0,0,385000.0,NaN,1,NaN
2,2021-01-01,10700000,4,13,3,85.0,12.0,55.540060,37.725112,3,0,142701.0,242543.0,50,681306.0
3,2021-01-01,3100000,3,5,3,82.0,9.0,44.608154,40.138381,0,0,385000.0,NaN,1,NaN
4,2021-01-01,2500000,2,3,1,30.0,9.0,44.738685,37.713668,3,2,353960.0,439378.0,23,1730985.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11358145,2021-12-31,6099000,4,9,3,65.0,0.0,56.041539,92.753133,0,0,660030.0,581436.0,24,857003.0
11358146,2021-12-31,2490000,1,10,2,56.9,0.0,55.169949,61.519210,0,0,454079.0,274414.0,74,1820769.0
11358147,2021-12-31,850000,2,2,2,37.0,5.0,55.946206,43.088179,0,0,606101.0,190983.0,52,958329.0
11358148,2021-12-31,4360000,5,5,1,36.0,9.0,61.256383,73.435919,0,0,628406.0,581702.0,86,2156710.0


Отобразим на карте координаты наших построек.

In [3]:
import folium
from IPython.display import display

map_df = df.loc[:1000]

m = folium.Map(location=[55.751244, 37.618423], zoom_start=10)

# Список точек с широтой и долготой
lats = map_df['geo_lat'].loc[:1000]
longs = map_df['geo_lon'].loc[:1000]
# Добавляем точки на карту
for point in zip(lats, longs):
    folium.Marker(
        location=[point[0], point[1]]
    ).add_to(m)

display(m)
m.save('map.html') # Сохраним ещё файл, чтобы можно было открыть через браузер

# Часть 1. Подготовим данные для обработки моделями машинного обучения.

**0.5 Балл**. География наших наблюдений в наборе данных крайне большая. Однако мы знаем, что стоимость квартир в Москве и Санкт-Петербурге намного выше, чем в среднем по России. Давайте сделаем признаки, который показывают, находится ли квартира в 20 килиметрах от центра Москвы или находится ли квартира в 20 килиметрах от центра Санкт-Петербурга.

Создайте два признака is_Moscow и is_Saint_Peterburg. Для нахождения расстояния по координатам используйте функцию haversine_distance.

In [ ]:
!pip install geopy
# Добавим данную библиотеку, чтобы взять оттуда координаты городов

In [4]:
from geopy.geocoders import Nominatim

def haversine_distance(lat1, lon1, lat2, lon2):

    EARTH_RAD = 6371 # Радиус Земли

    lat1, lon1, lat2, lon2 = map(np.radians, [lat1, lon1, lat2, lon2]) # Переводим в радианы для numpy

    # Вычисляем разницу
    d_lat = lat2 - lat1
    d_lon = lon2 - lon1

    distance = 2 * EARTH_RAD * np.arcsin(np.sqrt(np.sin(d_lat / 2) ** 2 + np.cos(lat1) * np.cos(lat2) * np.sin(d_lon / 2) ** 2)) # Вычислчем расстояние по формуле

    return distance

geoloc = Nominatim(user_agent="real_estate_analysis")

# Получаем координаты центров Москвы и Санкт-Петербурга
MSC_CENTER = geoloc.geocode("Москва, Россия")
SPB_CENTER = geoloc.geocode("Санкт-Петербург, Россия")

# Создаем необходимые столбцы и заполняем их значениями сразу (используем для хранения Int8, а не 64, чтобы уменьшить потребление памяти)
df['is_Moscow'] = (haversine_distance(df['geo_lat'], df['geo_lon'], MSC_CENTER.latitude, MSC_CENTER.longitude) <= 20).astype('Int8')
df['is_Saint_Peterburg'] = (haversine_distance(df['geo_lat'], df['geo_lon'], SPB_CENTER.latitude, SPB_CENTER.longitude) <= 20).astype('Int8')
df


,date,price,level,levels,rooms,area,kitchen_area,geo_lat,geo_lon,building_type,object_type,postal_code,street_id,id_region,house_id,is_Moscow,is_Saint_Peterburg
0,2021-01-01,2451300,15,31,1,30.3,0.0,56.780112,60.699355,0,2,620000.0,NaN,66,1632918.0,0,0
1,2021-01-01,1450000,5,5,1,33.0,6.0,44.608154,40.138381,0,0,385000.0,NaN,1,NaN,0,0
2,2021-01-01,10700000,4,13,3,85.0,12.0,55.540060,37.725112,3,0,142701.0,242543.0,50,681306.0,1,0
3,2021-01-01,3100000,3,5,3,82.0,9.0,44.608154,40.138381,0,0,385000.0,NaN,1,NaN,0,0
4,2021-01-01,2500000,2,3,1,30.0,9.0,44.738685,37.713668,3,2,353960.0,439378.0,23,1730985.0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11358145,2021-12-31,6099000,4,9,3,65.0,0.0,56.041539,92.753133,0,0,660030.0,581436.0,24,857003.0,0,0
11358146,2021-12-31,2490000,1,10,2,56.9,0.0,55.169949,61.519210,0,0,454079.0,274414.0,74,1820769.0,0,0
11358147,2021-12-31,850000,2,2,2,37.0,5.0,55.946206,43.088179,0,0,606101.0,190983.0,52,958329.0,0,0
11358148,2021-12-31,4360000,5,5,1,36.0,9.0,61.256383,73.435919,0,0,628406.0,581702.0,86,2156710.0,0,0


**0.5 Балла**. В нашем наборе данных есть признаки, которые мы теоретически можем использовать, например postal_code, но мы это будем делать в рамках домашней работы очень-очень долго. Поэтому предлагается удалить ненужные признаки из датафрейма.

Удалим geo_lat,	geo_lon,	object_type,	postal_code,	street_id,	id_region,	house_id.

In [5]:
df = df.drop(['geo_lat', 'geo_lon', 'object_type', 'postal_code', 'street_id', 'id_region', 'house_id'], axis=1)
df

,date,price,level,levels,rooms,area,kitchen_area,building_type,is_Moscow,is_Saint_Peterburg
0,2021-01-01,2451300,15,31,1,30.3,0.0,0,0,0
1,2021-01-01,1450000,5,5,1,33.0,6.0,0,0,0
2,2021-01-01,10700000,4,13,3,85.0,12.0,3,1,0
3,2021-01-01,3100000,3,5,3,82.0,9.0,0,0,0
4,2021-01-01,2500000,2,3,1,30.0,9.0,3,0,0
...,...,...,...,...,...,...,...,...,...,...
11358145,2021-12-31,6099000,4,9,3,65.0,0.0,0,0,0
11358146,2021-12-31,2490000,1,10,2,56.9,0.0,0,0,0
11358147,2021-12-31,850000,2,2,2,37.0,5.0,0,0,0
11358148,2021-12-31,4360000,5,5,1,36.0,9.0,0,0,0


**0.5 Балл**. Для начала Вам предлагается проанализировать Ваши оставшиеся признаки (колонки) в наборе данных. Какие колонки категориальные? Какие числовые?

Категориальные: date, building_type, is_Moscow, is_Saint_Peterburg, rooms (да, этот признак тоже отнесём сюда, так как количество комнат может влиять на воприятие квартиры (для кого она и тп), поэтому признак влияет, скажем так, на тип квартиры).

Числовые: price, level, levels, area, kitchen_area.

Давайте закодируем категориальные признаки с помощью OneHot-Encoding. https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OneHotEncoder.html

In [6]:
from sklearn.preprocessing import OneHotEncoder

df['date'] = pd.to_datetime(df['date']).astype('int64') // 10**9 // 86400 # Переведём дату в количество дней с определённого момента (01.01.1970)

categorical_features = ['building_type', 'rooms'] # Колонки для кодирования

encoder = OneHotEncoder(sparse_output=False, drop='first') # Будем убирать один из столбцов после кодировки, так как его значение уже понятно из значения других, то есть мы экономим память и стараемся избежать мультиколлинеарности

encoded_array = encoder.fit_transform(df[categorical_features])

encoded_columns = encoder.get_feature_names_out(categorical_features)

df_encoded = pd.DataFrame(encoded_array, columns=encoded_columns, index=df.index).astype('Int8') # Используем также Int8

df = pd.concat([df.drop(categorical_features, axis=1), df_encoded], axis=1) # Добавляем наши новые закодированные признаки, удаляя старые
# Колонки про Москву и Санкт-Петербург уже закодированы
df

,date,price,level,levels,area,kitchen_area,is_Moscow,is_Saint_Peterburg,building_type_1,building_type_2,...,building_type_6,rooms_1,rooms_2,rooms_3,rooms_4,rooms_5,rooms_6,rooms_7,rooms_8,rooms_9
0,18628,2451300,15,31,30.3,0.0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
1,18628,1450000,5,5,33.0,6.0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
2,18628,10700000,4,13,85.0,12.0,1,0,0,0,...,0,0,0,1,0,0,0,0,0,0
3,18628,3100000,3,5,82.0,9.0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
4,18628,2500000,2,3,30.0,9.0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11358145,18992,6099000,4,9,65.0,0.0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
11358146,18992,2490000,1,10,56.9,0.0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
11358147,18992,850000,2,2,37.0,5.0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
11358148,18992,4360000,5,5,36.0,9.0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0


**0.5 Балл**. Поработаем с числовыми признаками:


1.   Добавьте в ваш датасет два признака: количество дней со дня первого наблюдения (разница между датами объявлений). Возможно, для предсказания цены не так важен этаж, как важно отношение этажа квартиры на количество этажей в доме, добавьте этот признак. После добавления нового признака колонку date можно удалить.
2.   Числовые признаки могут иметь разные порядки. Давайте отнормируем числовые признаки с помощью StandartScaller https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html.



In [7]:
df['days'] = df['date'] - df['date'].min() # Высчитываем число дней со дня первого наблюдения
df = df.drop(['date'], axis=1)

df['relative_level'] = df['level'] / df['levels'] # Отношение номера этажа к количеству этажей в доме

df = df.drop(['level', 'levels'], axis=1) # Удалим номер этажа и количество этажей в доме, так как мы уже используем их отношение

df


,price,area,kitchen_area,is_Moscow,is_Saint_Peterburg,building_type_1,building_type_2,building_type_3,building_type_4,building_type_5,...,rooms_2,rooms_3,rooms_4,rooms_5,rooms_6,rooms_7,rooms_8,rooms_9,days,relative_level
0,2451300,30.3,0.0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0.483871
1,1450000,33.0,6.0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1.000000
2,10700000,85.0,12.0,1,0,0,0,1,0,0,...,0,1,0,0,0,0,0,0,0,0.307692
3,3100000,82.0,9.0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0.600000
4,2500000,30.0,9.0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0.666667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11358145,6099000,65.0,0.0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,364,0.444444
11358146,2490000,56.9,0.0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,364,0.100000
11358147,850000,37.0,5.0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,364,1.000000
11358148,4360000,36.0,9.0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,364,1.000000


In [8]:
from sklearn.preprocessing import StandardScaler

# Колонки для нормировки
num_features = ['area', 'kitchen_area', 'days', 'relative_level']

scaler = StandardScaler()
df = df[~np.isinf(df['relative_level'])] # Удалим строки, где бесконечные значения у указанного признака


df[num_features] = scaler.fit_transform(df[num_features])

df

C:\Users\polin\AppData\Local\Temp\ipykernel_21860\1343723713.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[num_features] = scaler.fit_transform(df[num_features])


,price,area,kitchen_area,is_Moscow,is_Saint_Peterburg,building_type_1,building_type_2,building_type_3,building_type_4,building_type_5,...,rooms_2,rooms_3,rooms_4,rooms_5,rooms_6,rooms_7,rooms_8,rooms_9,days,relative_level
0,2451300,-0.840594,0.082513,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,-1.858321,-0.308580
1,1450000,-0.741061,0.267578,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,-1.858321,1.452761
2,10700000,1.175873,0.452643,1,0,0,0,1,0,0,...,0,1,0,0,0,0,0,0,-1.858321,-0.909807
3,3100000,1.065280,0.360110,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,-1.858321,0.087722
4,2500000,-0.851653,0.360110,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,-1.858321,0.315229
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11358145,6099000,0.438591,0.082513,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,1.752638,-0.443127
11358146,2490000,0.139991,0.082513,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,1.752638,-1.618577
11358147,850000,-0.593605,0.236734,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,1.752638,1.452761
11358148,4360000,-0.630469,0.360110,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1.752638,1.452761


**2 Балла**. Реализуйте класс KNNRegressor, который должен делать регрессию методом k ближайших соседей.

In [ ]:
!pip install tqdm 
# Это библиотека для отслеживания прогресса выполнения. Добавил её, чтобы следить за прогрессом выполнения KNN, так как данных много. 
# Это помогло убедится что обычный KNN будет работать очень долго (11М строк в датасете), поэтому нужно оптимизировать.

In [9]:
from tqdm.auto import tqdm
class KNNRegressor:
    def __init__(self, n_neighbors=5, metric='euclidean', batch_size=1000, sample_size=100000, rand_state=52):
      self.n_neighbors = n_neighbors
      self.metric = metric
      self.batch_size = batch_size # Будем производить вычисления по батчам (группам), чтобы ускорить, но при этом не перегружать память
      self.sample_size = sample_size # Размер части дата сета, на котором будет "обучаться" модель. 
      # Нужно, так как размер всей выборки большой (8М+) и делать вычисления для всех данных ооочень долго. Но при достаточном размере закономерности сохраняются и ответ должен сильно ухудшиться 
      
      self.rand_state = rand_state # Номер сида для рандомного взятия части выборки (помогает фиксировать рандомное взятие, то есть при одинаковом значении переменной и общей выбоки, значения будут те же)

    def fit(self, X, y):
      self.X_train = np.array(X, dtype=np.float32)
      self.y_train = np.array(y, dtype=np.float32)

      # Вычисляем выборку, на которой будет обучени
      if len(self.X_train) > self.sample_size:
        self.X_train_sample, self.y_train_sample = self._sample(self.X_train, self.y_train)
      else:
        self.X_train_sample, self.y_train_sample = self.X_train, self.y_train

      return self

    def predict(self, X):
      X = np.array(X, dtype=np.float32)
      n = X.shape[0]
      y_pred = np.zeros(n, dtype=np.float32)

      for i in tqdm(range(0, n, self.batch_size)): # Перебираем данные, но с шагом по размеру батча. Также сделаем отслеживание прогресса
        end = min(i + self.batch_size, n) # Выбираем конечную точку батча (чтобы не перескочить конец данных)
        dists = self._calc_dists(X[i:end]) # Считаем расстояния

        neighbors = np.argpartition(dists, self.n_neighbors, axis=1)[:, :self.n_neighbors] # Выбираем ближайших соседей

        y_pred[i:end] = np.mean(self.y_train_sample[neighbors], axis=1) # Вычисляем среденее значение по соседям

      return y_pred

    def _calc_dists(self, X_batch): # Подсчёт расстояний
        # Сделаем обработку метрики, чтобы при необходимости можно было добавить другие
        if self.metric == 'euclidean':
          if not hasattr(self, 'X_train_sq'): # Проверка на существование переменной
            self.X_train_sq = np.sum(self.X_train_sample ** 2, axis=1)

          X_batch_sq = np.sum(X_batch ** 2, axis=1, keepdims=True)

          dot_product = np.dot(X_batch, self.X_train_sample.T)

          dists = X_batch_sq - 2 * dot_product + self.X_train_sq # (a - b) ^ 2 = a ^ 2 - 2 * a * b + b ^ 2 

        return dists # Не будем брать корень из расстояний, будем хранить квадраты. На выбор соседей это не повлияет (очевидно)
    
    def _sample(self, X_train, y_train): # Выбор сэмпла для обучения
        np.random.seed(self.rand_state)

        n = len(X_train)
        ids = np.random.choice(n, size=self.sample_size, replace=False)

        return X_train[ids], y_train[ids]


**3 Балла**. Реализуйте класс LinearRegression, поддерживающий обучение градиентными спусками SGD, Momentum, AdaGrad. Используйте градиент для оптимизации функции потерь MSE.

In [10]:
class LinearRegression:
    def __init__(self, learning_rate=0.01, optimization='SGD', epsilon=1e-8, decay_rate=0.9, max_iter=1000, batch_size=1):
        self.learning_rate = learning_rate
        self.optimization = optimization
        self.epsilon = epsilon
        self.decay_rate = decay_rate
        self.max_iter = max_iter
        self.weights = None
        self.bias = None
        self.loss_history = [] # Будем сохранять историю ошибок, чтобы можно было выйти из цикла итераций и не выполнять их дельше, если изменений особо не происходит
        self.batch_size = batch_size # Размер батча. Добавим возможность менять размер батча (изначально 1 для SGD)

    def fit(self, X, y):
        X = np.array(X, dtype=np.float64)
        y = np.array(y, dtype=np.float64)

        n_samples, m_features = X.shape
        self.weights = np.zeros(m_features, dtype=np.float64)
        self.bias = 0

        if self.optimization == 'Momentum':
            # Скорости изменения
            v_w = np.zeros(m_features, dtype=np.float64)
            v_b = 0.0

        elif self.optimization == 'AdaGrad':
            # Переменные для квадратов градиентов
            c_w = np.zeros(m_features, dtype=np.float64)
            c_b = 0.0

        for i in range(self.max_iter):
            # Выделяем батч, чтобы не работать со всеми данными (для оптимизации)
            ids = np.random.choice(n_samples, self.batch_size, replace=False)
            X_batch = X[ids]
            y_batch = y[ids]
        
            y_pred = np.dot(X_batch, self.weights) + self.bias
            error = y_pred - y_batch

            # Градиенты функции потерь MSE 
            grad_w = (2/self.batch_size) * np.dot(X_batch.T, error)
            grad_b = (2/self.batch_size) * np.sum(error)

            if self.optimization == 'SGD':
                # Шаг
                self.weights -= self.learning_rate * grad_w
                self.bias -= self.learning_rate * grad_b

            elif self.optimization == 'Momentum':
                # Шаг с учётом скорости
                v_w = self.decay_rate * v_w + self.learning_rate * grad_w
                v_b = self.decay_rate * v_b + self.learning_rate * grad_b
                self.weights -= v_w
                self.bias -= v_b

            elif self.optimization == 'AdaGrad':
                # Шаг с учётом обновления размера шага и их адаптируемости под параметры
                c_w += grad_w ** 2
                c_b += grad_b ** 2

                adjusted_lr_w = self.learning_rate / (np.sqrt(c_w) + self.epsilon)
                adjusted_lr_b = self.learning_rate / (np.sqrt(c_b) + self.epsilon)

                self.weights -= adjusted_lr_w * grad_w
                self.bias -= adjusted_lr_b * grad_b

            loss = np.mean(error ** 2)
            self.loss_history.append(loss) # Сохраняем истьорию ошибок

            if i > 20 and abs(self.loss_history[-2] - loss) < 1e-4: # Если изменения ошибок не большие, при этом уже сделаны шаги, то можно выйти из цикла, так как нет особо смысла выполнять итерации
                break

        return self



    def predict(self, X):
        return np.dot(np.array(X), self.weights) + self.bias

# Часть 2. Эксперименты с моделями машинного обучения.

**3 Балла**. Проведите эксперименты с написанными Вами методами машинного обучения. Выделите обучающую и тестовую выборки в отношении 0,8 и 0,2 соответственно. Измерьте ошибку MSE, MAE, RMSE. Заиспользуйте методы KNNRegressor и LinearRegression из библиотеки sklearn, сравните качество Ваших решений и библиотечных.

In [11]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
from sklearn.neighbors import KNeighborsRegressor as SklearnKNNR
from sklearn.linear_model import SGDRegressor as SklearnLR # Воспользуемся именно этим методом, так наши модели тоже с GD + работает быстрее, чем обычная LR (обычная считала бы долго на таком объёме данных (я пробовал))

df = df.dropna() # Выбрасываем строки где есть пустые значения
# # Отделяем целевую переменную
X = df.drop(columns=['price'])
y = df['price']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=52) # Разделение на обучающую и тестовую выборки (0,8 и 0,2 соответственно)

# Создаём модели
my_knn = KNNRegressor(batch_size=100, sample_size = 50000)
my_lr_s = LinearRegression(learning_rate=0.1)
my_lr_m = LinearRegression(learning_rate=0.1, optimization='Momentum')
my_lr_a = LinearRegression(learning_rate=0.1, optimization='AdaGrad')
 
sklearn_knn = SklearnKNNR(n_neighbors=5)
sklearn_lr = SklearnLR(max_iter=1000, tol=1e-4,  learning_rate='constant', eta0=0.01, random_state=52)






In [18]:
my_knn.fit(X_train, y_train) # Обучение
y_pred = my_knn.predict(X_test) # Предсказание

# Вычисление метрик для ошибок и вывод
mse = np.mean((y_test - y_pred) ** 2)
mae = mean_absolute_error(y_test, y_pred)
print('KNN')
print(f'MSE: {mse}')
print(f'MAE: {mae}')
print(f'RMSE: {np.sqrt(mse)}')

  0%|          | 0/22712 [00:00<?, ?it/s]

KNN
MSE: 2271034002234307.0
MAE: 3541998.0
RMSE: 47655366.98247436


In [21]:
my_lr_s.fit(X_train, y_train)
y_pred = my_lr_s.predict(X_test)

mse = np.mean((y_test - y_pred) ** 2)
mae = mean_absolute_error(y_test, y_pred)
print('LR_SGD')
print(f'MSE: {mse}')
print(f'MAE: {mae}')
print(f'RMSE: {np.sqrt(mse)}')


LR_SGD
MSE: 3.313482237111947e+23
MAE: 439487962229.27496
RMSE: 575628546643.749


In [22]:
my_lr_m.fit(X_train, y_train)
y_pred = my_lr_m.predict(X_test)

mse = np.mean((y_test - y_pred) ** 2)
mae = mean_absolute_error(y_test, y_pred)
print('LR_Momentum')
print(f'MSE: {mse}')
print(f'MAE: {mae}')
print(f'RMSE: {np.sqrt(mse)}')

LR_Momentum
MSE: 1.6499286912806173e+173
MAE: 3.2153104605489696e+86
RMSE: 4.061931426403722e+86


In [ ]:
my_lr_a.fit(X_train, y_train)
y_pred = my_lr_a.predict(X_test)

mse = np.mean((y_test - y_pred) ** 2)
mae = mean_absolute_error(y_test, y_pred)
print('LR_AdaGrad')
print(f'MSE: {mse}')
print(f'MAE: {mae}')
print(f'RMSE: {np.sqrt(mse)}')
# Эта часть кода запускалась и отрабатывала нормально без ошибок (в том числе и с памятью), 
# просто я перепроверял всё и случайно её перезапустил, и вначале вылетела ошибка с памятью, а потом начались проблемы с подключением к виртуальному окружению.
# Я пытался их решить (в том числе пересозавал окружение), но пока не вышло... (как-то в этот раз много проблем с окружениями)) ). 
# До, как я и сказал, всё отрабатывало. Ошибка (значение) была схожа с ошибкой при отработке соответствующей модели из sklearn (см. ниже) 

MemoryError: Unable to allocate 1.42 GiB for an array with shape (9084798, 21) and data type float64

In [ ]:
# Возьмём только часть тестовой выборки, так как работать со всей очень долго
X_sample = X_train.sample(n=50000, random_state=52)
y_sample = y_train.loc[X_sample.index]

sklearn_knn.fit(X_sample, y_sample)
y_pred = sklearn_knn.predict(X_test)

mse = np.mean((y_test - y_pred) ** 2)
mae = mean_absolute_error(y_test, y_pred)
print('Sklearn_KNN')
print(f'MSE: {mse}')
print(f'MAE: {mae}')
print(f'RMSE: {np.sqrt(mse)}')

Sklearn_KNN
MSE: 2271033939819380.0
MAE: 3541986.718284431
RMSE: 47655366.32761708


In [15]:
sklearn_lr.fit(X_train, y_train)
y_pred = sklearn_lr.predict(X_test)

mse = np.mean((y_test - y_pred) ** 2)
mae = mean_absolute_error(y_test, y_pred)
print('Sklearn_LR')
print(f'MSE: {mse}')
print(f'MAE: {mae}')
print(f'RMSE: {np.sqrt(mse)}')

Sklearn_LR
MSE: 2001822630941415.2
MAE: 7073915.2964098295
RMSE: 44741732.54291138


Ошибки реализованных моделей схожи с библиотечными (в линейной регрессии первые две оптимизации схожи только если увеличить размер батча (например 5000), при размере равном 1 ошибки сильно больше), но работают они дольше, так как библиотечные более оптимизированы. Ошибки у всех моделей довольно большие, возможно из-за проблем с данными и выбросов.